<a href="https://colab.research.google.com/github/ahreum239/Test1/blob/main/Main_Comment_Analysis_Data_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
# Imports
import pandas as pd
import numpy as np
import json
import re
from datetime import datetime, timedelta
from google.colab import drive
import os

In [ ]:
# Mount Google Drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load the saved data

In [ ]:
# Constants
project_dir = '/content/drive/MyDrive/AI Summary Project/Model Free analysis/Comment analysis'
sentiment_default = 0.5 # Replace sentiment NA as 0.5

# Load JSON data
def load_json(filepath):
    """Load JSON data from a file."""
    return pd.read_json(filepath, lines=True)

The matched videos

In [ ]:
# Load Video Matched Data
matched_pre_path = f"{os.path.dirname(project_dir)}/data_before_group.json"
matched_exp_path = f"{os.path.dirname(project_dir)}/data_before_group_exp.json"
matched = pd.concat([load_json(matched_pre_path), load_json(matched_exp_path)])

In [ ]:
print(len(matched))

5330


In [ ]:
print(matched.groupby('treat').size())

treat
0    2665
1    2665
dtype: int64


In [ ]:
print(matched.groupby(['treat']).mid.nunique())

treat
0    301
1    301
Name: mid, dtype: int64


In [ ]:
# Load Matched Video Pairs Data
pairs_pre_path = f"{os.path.dirname(project_dir)}/matched_pairs.json"
pairs_exp_path = f"{os.path.dirname(project_dir)}/matched_pairs_exp.json"
pairsT = pd.concat([load_json(pairs_pre_path), load_json(pairs_exp_path)])

In [ ]:
print(len(pairsT))

2665


In [ ]:
print(matched.iloc[0])

mid                                    546195
namd                                      老番茄
title                              老番茄一秒钟治好鼻炎
vid                               T2iSXqiCBLo
date                      2023-12-09 00:00:00
length                                   1347
description    游戏：《瓦力欧制造：超级舞动》\n这个游戏太抽象了哈哈哈哈哈
view_n                                  21384
like_n                                    602
cmt_n                                      18
treat                                       0
period                                      6
Name: 0, dtype: object


In [ ]:
print(pairsT.iloc[0])

yvid                 rwi2zyZUzww
bvid                BV1184y1S7c1
ratio                   0.976744
ytitle    细读经典你管这叫喜剧 开心麻花巅峰之作驴得水
btitle     细读经典你管这叫喜剧开心麻花巅峰之作驴得水
Name: 0, dtype: object


Comment data

In [ ]:
# Load and Process Comment Data
bcmt_path = f"{project_dir}/bili_sentiment_expT.json"
ycmt_path = f"{project_dir}/youtube_sentiment_rep_id.json"
bcmt_data = load_json(bcmt_path).fillna({'sentiment': sentiment_default})
ycmt_data = load_json(ycmt_path).fillna({'sentiment': sentiment_default})

In [ ]:
print(bcmt_data.iloc[0])

reply_id                      195136516640
user_name                        我抬手就是一个Z炮
sex                                     保密
level                                    5
content      虚拟主播的演员，一般被叫做“中之人”。内胆这个称呼非常少见
time                            1700441095
like_num                                12
reply_num                              0.0
reply_to                               NaN
vid                           BV1jw411K7eJ
sentiment                         0.619635
Name: 0, dtype: object


In [ ]:
print(ycmt_data.iloc[0])

id                   UgzgfJbSMrhCjwZQue54AaABAg
type                            topLevelComment
channelId              UCSN9fvRC2vzI1H6QCWZDZNg
videoId                             hEMzzL8n-Os
totalReplyCount                             0.0
isPublic                                    1.0
textOriginal                            1:44 盜片
authorDisplayName              @user-yz9gz4hq3l
viewerRating                               none
likeCount                                     0
publishedAt                2024-01-13T06:53:40Z
updatedAt                  2024-01-13T06:53:40Z
parentId                                   None
sentiment                                   0.5
Name: 0, dtype: object


# Matched Videos Only

Filter comments for matched videos only

In [ ]:
"""Filter comments for matched videos only."""
def filter_comments_for_videos(df, video_column, video_ids):
    # Function to filter comments for matched videos only
    return df[df[video_column].isin(video_ids)]

In [ ]:
# Filter comments for matched videos only
y_videos = matched[matched['treat'] == 0]['vid'].unique()
b_videos = matched[matched['treat'] == 1]['vid'].unique()
ycmt_filtered = filter_comments_for_videos(ycmt_data, 'videoId', y_videos)
bcmt_filtered = filter_comments_for_videos(bcmt_data, 'vid', b_videos)

In [ ]:
print(len(ycmt_filtered))
print(len(bcmt_filtered))

# Not all videos (n=2665) has comment data
print((ycmt_filtered.videoId.nunique()))
print((bcmt_filtered.vid.nunique()))

103392
1664616
2312
2651


# Filter by Time Window

In [ ]:
# Constants
min_date, max_date = '2023-09-04', '2023-12-10'

def convert_to_datetime(df, column, time_format=None, unit=None):
    # Convert timestamps to datetime with UTC
    df[column] = pd.to_datetime(df[column], format=time_format, unit=unit, utc=True)

def filter_time_window(df, date_col, start=min_date, end=max_date):
    """Filter DataFrame by date range."""
    df.loc[:, 'cmt_date'] = df[date_col].dt.strftime('%Y-%m-%d')
    return df[(df['cmt_date'] >= start) & (df['cmt_date'] <= end)]

# Process ycmt and bcmt DataFrames
def filter_comments_for_times(ycmt, bcmt):

    # Convert timestamps to datetime with appropriate formats
    convert_to_datetime(ycmt, 'publishedAt', time_format='%Y-%m-%dT%H:%M:%SZ')
    convert_to_datetime(bcmt, 'time', unit='s')

    # Rename datetime columns for consistency
    ycmt.rename(columns={'publishedAt': 'cmt_timestamp'}, inplace=True)
    bcmt.rename(columns={'time': 'cmt_timestamp'}, inplace=True)

    # Filter by date range and add period column
    ycmt = filter_time_window(ycmt, 'cmt_timestamp')
    bcmt = filter_time_window(bcmt, 'cmt_timestamp')

    return ycmt, bcmt

In [ ]:
# Run the processing function
ycmt, bcmt = filter_comments_for_times(ycmt_filtered, bcmt_filtered)

Get Period Variable

In [ ]:
"""Add period columns based on weekly intervals."""

def make_timezone_naive(df, timestamp_column):
    # Convert timestamp columns to timezone-naive (removing timezone information)
    df[timestamp_column] = df[timestamp_column].dt.tz_convert(None)

def calculate_period(date):
    # Extract week number in the format 'YYYY-WW' and adjust based on starting period
    iso_week = date.strftime('%G-%V')
    return int(iso_week[-2:]) - 43

def add_period_column(df, timestamp_column):
    # Add 'period' column based on weekly intervals
    df.loc[:, 'period'] = df[timestamp_column].dt.to_period('W').apply(lambda x: calculate_period(x.start_time))

# Process ycmt and bcmt DataFrames
def process_comments(ycmt, bcmt):

    for df in [ycmt, bcmt]:
        make_timezone_naive(df, 'cmt_timestamp')
        add_period_column(df, 'cmt_timestamp')

    return ycmt, bcmt

In [ ]:
# Run the processing function
ycmt_pr, bcmt_pr = process_comments(ycmt, bcmt)

In [ ]:
print(len(ycmt_filtered))
print(len(bcmt_filtered))

# Not all videos (n=2665) has comment data within limited time window
print((ycmt_pr.videoId.nunique()))
print((bcmt_pr.vid.nunique()))

103392
1664616
2173
2550


# Concat Data at Comment-Video Level

In [ ]:
# Define selected columns and standardized column names for YouTube and Bilibili data
selected_columns = {
    "bilibili": ['reply_id', 'vid', 'user_name', 'like_num', 'reply_num', 'sentiment', 'content', 'cmt_timestamp', 'period'],
    "youtube": ['id', 'videoId', 'authorDisplayName', 'likeCount', 'totalReplyCount', 'sentiment', 'textOriginal', 'cmt_timestamp', 'period']
}
standard_columns = ['commentid', 'vid', 'cmt_user', 'cmt_likes', 'cmt_replies', 'sentiment', 'comment', 'cmt_timestamp', 'period']

# Create DataFrames with standardized columns
bcmt_df = bcmt_pr[selected_columns["bilibili"]].copy()
ycmt_df = ycmt_pr[selected_columns["youtube"]].copy()
bcmt_df.columns = ycmt_df.columns = standard_columns

# Assign treatment indicator: 1 for Bilibili, 0 for YouTube
bcmt_df['treat'] = 1
ycmt_df['treat'] = 0

# Concatenate both DataFrames into a single DataFrame
cmt_df = pd.concat([bcmt_df, ycmt_df], ignore_index=True)

# channel_id (mid)
mid_vid = matched[['mid','vid']].drop_duplicates()
cmt_df = pd.merge(cmt_df, mid_vid, on='vid', how='left')

# Timestamp
vid_timestamp = matched[['vid','period']]
vid_timestamp.columns = ['vid','vd_period']
cmt_df = pd.merge(cmt_df, vid_timestamp, on='vid', how='left')

In [ ]:
# Filters for videos that are present in both YouTube and Bilibili datasets
common_videos = pairsT[pairsT['yvid'].isin(ycmt_df['vid']) & pairsT['bvid'].isin(bcmt_df['vid'])]
common_yvids = common_videos['yvid']
common_bvids = common_videos['bvid']

# Filter for common video records and drop rows with missing comments
cmt_df_pr = cmt_df[cmt_df['vid'].isin(common_yvids) | cmt_df['vid'].isin(common_bvids)]

# Drop records with missing comments
cmt_df_pr = cmt_df_pr.dropna(subset=['comment'])

In [ ]:
# Unique videos per treatment group
print(cmt_df.groupby('treat')['vid'].nunique().reset_index(name='unique_videos'))
print(cmt_df_pr.groupby('treat')['vid'].nunique().reset_index(name='unique_videos'))

# Unique channels per treatment group
print(cmt_df.groupby('treat')['mid'].nunique().reset_index(name='unique_channels'))
print(cmt_df_pr.groupby('treat')['mid'].nunique().reset_index(name='unique_channels'))

   treat  unique_videos
0      0           2173
1      1           2550
   treat  unique_videos
0      0           2162
1      1           2162
   treat  unique_channels
0      0              281
1      1              298
   treat  unique_channels
0      0              281
1      1              281


In [ ]:
def apply_sentiment_categories(df, sentiment_col='sentiment'):
    """Add sentiment categories based on sentiment score."""
    df['pos_cmt'] = (df[sentiment_col] >= 0.6).astype(int)
    df['neg_cmt'] = (df[sentiment_col] <= 0.4).astype(int)
    df['neu_cmt'] = ((df[sentiment_col] > 0.4) & (df[sentiment_col] < 0.6)).astype(int)
    return df

In [ ]:
cmt_df_pr = apply_sentiment_categories(cmt_df_pr, sentiment_col='sentiment')

In [ ]:
print(cmt_df_pr.iloc[0])

commentid                                    193843602800
vid                                          BV1sj411a77r
cmt_user                                         awiwicat
cmt_likes                                              56
cmt_replies                                           1.0
sentiment                                        0.527836
comment          哇塞，一分钟我就刷新了，这种老恐怖片有种年代的单调平实感觉，这种程度都吓不到人了
cmt_timestamp                         2023-11-10 14:45:01
period                                                  2
treat                                                   1
mid                                               2390247
vd_period                                               2
pos_cmt                                                 0
neg_cmt                                                 0
neu_cmt                                                 1
Name: 85, dtype: object


# Handle Outliers

In [ ]:
def filter_channels_by_upload_frequency(df, min_weeks=5):
    """Filter for channels with a minimum video upload frequency."""
    channel_counts = (
        df.groupby(['treat', 'mid'])['vd_period']
        .nunique()
        .reset_index(name='vd_count')
    )
    return channel_counts[channel_counts['vd_count'] >= min_weeks]

def filter_channels_present_on_both_platforms(df, valid_mids):
    """Ensure channels are present on both platforms (YouTube and Bilibili)."""
    filtered_df = valid_mids.copy()
    platform_check = (
        filtered_df.groupby('mid')
        .filter(lambda x: x['treat'].nunique() == 2)['mid']
        .unique()
    )
    return df[df['mid'].isin(platform_check)]

def filter_for_period_existence(df, period_column='vd_period'):
    """Ensure both 'before' and 'after' periods exist for each channel-platform pair."""
    df['after'] = (df[period_column] >= 0).astype(int)
    period_counts = df.groupby(['treat', 'mid', 'after']).size().unstack(fill_value=0)
    valid_pairs = period_counts[(period_counts[0] > 0) & (period_counts[1] > 0)].index
    return df[df.set_index(['treat', 'mid']).index.isin(valid_pairs)]

In [ ]:
def filter_high_discrepancy_mids(matchedT, valid_mids, cmt_text_df, discrepancy_threshold=5):
    """
    Filters out 'mid' values with high discrepancies in user engagement metrics (view_n) between treatment groups.
    """
    filtered_T = matchedT[matchedT.mid.isin(valid_mids.mid.unique())]

    comparison = (
        filtered_T
        .groupby(['treat', 'mid'])
        .agg({'view_n': 'mean'})  # Add other metrics as needed (e.g., 'like_n', 'cmt_n', 'length')
        .reset_index()
    )
    comparison_pivot = comparison.pivot(index='mid', columns='treat', values=['view_n'])
    comparison_pivot.columns = [f'{metric}_treat{treat}' for metric, treat in comparison_pivot.columns] # Flatten the column MultiIndex
    comparison_pivot['view_diff'] = (comparison_pivot['view_n_treat1'] - comparison_pivot['view_n_treat0']).abs() # Calculate the absolute discrepancies
    avg_view_diff = comparison_pivot['view_diff'].mean()

    # Filter out these high-discrepancy mids from the main DataFrame
    discrepant_mids = comparison_pivot[
        (comparison_pivot['view_diff'] > avg_view_diff * discrepancy_threshold)
    ].index.tolist()

    filtered_cmt_text_df = cmt_text_df[~cmt_text_df.mid.isin(discrepant_mids)]
    return filtered_cmt_text_df

In [ ]:
valid_mids = filter_channels_by_upload_frequency(cmt_df_pr)

cmt_df1 = filter_channels_present_on_both_platforms(cmt_df_pr, valid_mids)
cmt_df2 = filter_for_period_existence(cmt_df1)
cmt_df3 = filter_high_discrepancy_mids(matched, valid_mids, cmt_df2, discrepancy_threshold=5)

<ipython-input-260-036a5c9fa601>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['after'] = (df[period_column] >= 0).astype(int)


In [ ]:
# Debugging information
print(f"Initial merged DataFrame length: {len(cmt_df_pr)}")
print(f"Filtered channels_present_on_both_platforms: {len(cmt_df1)}")
print(f"Filtered for_period_existence: {len(cmt_df2)}")
print(f"Final filter_high_discrepancy_mids: {len(cmt_df3)}")

print(f"Unique video IDs in filtered DataFrame: {len(cmt_df3['vid'].unique())}")
print(f"Unique mids by treatment after filtering: {cmt_df3.groupby('treat')['mid'].nunique()}")

Initial merged DataFrame length: 1467956
Filtered channels_present_on_both_platforms: 1167642
Filtered for_period_existence: 1141127
Final filter_high_discrepancy_mids: 1105360
Unique video IDs in filtered DataFrame: 3324
Unique mids by treatment after filtering: treat
0    127
1    128
Name: mid, dtype: int64


In [ ]:
print(cmt_df3.iloc[0])

commentid               196719582208
vid                     BV1Je411Z7ti
cmt_user                      涛子先生先生
cmt_likes                        964
cmt_replies                     49.0
sentiment                   0.494229
comment                      我这样不香吗？
cmt_timestamp    2023-12-01 10:50:05
period                             5
treat                              1
mid                         27899754
vd_period                          5
pos_cmt                            0
neg_cmt                            0
neu_cmt                            1
after                              1
Name: 277, dtype: object


# Save JSON data

In [ ]:
# # Refer https://colab.research.google.com/drive/1NkpEjRs081qCUV4agH_QQ4iqgwPOa6Ql#scrollTo=nxvBzhq_-b1Y
# # Export the list to a JSON file
# with open('cmt_text_df_filtered.json', 'w') as f:
#     # Convert the DataFrame to a JSON-serializable format (e.g., dictionary)
#     json.dump(cmt_df3.to_dict(orient='records'), f, default=str) # Added default=str to handle Timestamp objects